In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
import a_mps, b_model, c_tebd, tfi_exact

In [ ]:
L = 14
J = 1.0
g = 1.5
up = a_mps.init_spinup_MPS(L)
m = b_model.TFIModel(L=L, J=J , g=g)
print(f"sigma z: {up.site_expectation_value(m.sigmaz)}")
print(f"sigma x: {up.site_expectation_value(m.sigmax)}")

In [ ]:
right = a_mps.init_spinright_MPS(L)
print(f"sigma z: {right.site_expectation_value(m.sigmaz)}")
print(f"sigma x: {right.site_expectation_value(m.sigmax)}")

In [ ]:
L = 14
J = 1
gs = [0.5, 1.0, 1.5]
up = a_mps.init_spinup_MPS(L)
right = a_mps.init_spinright_MPS(L)

for g in gs:
    m = b_model.TFIModel(L=L, J=J, g=g)
    print(f"{g = :.1f}: {m.energy(up) = :.2f}")
    print(f"{g = :.1f}: {m.energy(right) = :.2f}")

In [ ]:
%run c_tebd

In [ ]:
def global_quench(L: int, dt: float, chi_max: int, eps: float) -> tuple[float, float]:
    """(magnetization, entropy)"""
    J = 1.0
    g = 1.5
    N_steps = 10*J / dt
    m = b_model.TFIModel(L=L, J=J, g=g)
    up = a_mps.init_spinup_MPS(L)
    bonds = c_tebd.calc_U_bonds(m, 1j * dt)
    c_tebd.run_TEBD(up, bonds, N_steps, chi_max, eps)
    M = sum(up.site_expectation_value(m.sigmaz))
    S = up.entanglement_entropy()[L//2]
